In [1]:
import urllib2, os, sys, subprocess
import googleapiclient.discovery
import os, sys
import random
import time 
import paramiko

In [2]:
def read_instance_properties():
    #compute-nodes: list of other instances that belong to cluster. Try using series-[1-100] notation
    #name of slurm-master 
    #machine type, can we get that from /proc/cpuinfo? 
    pass

In [23]:
def num_cpus():
    try:
        np=subprocess.check_output('cat /proc/cpuinfo | grep processor | wc -l', shell=True)
        p = int(np)
        return p
    except:
        p = 1 
        return p 

In [19]:
def machine_type(m):
    #Cat proc/cpuinfo, and some memory ? 
    #cpus = num_cpus()
    cpus = int(m.split('-')[-1])
    machine = {'sockets': 1, 'cores': cpus/2, 'threads': 2, 'memory': 1000}
    return machine 
#TODO: Convert to string type so that we can use it? 
    

In [20]:
project='first-220321'
zone='us-east1-b'
compute = googleapiclient.discovery.build('compute', 'v1')

slurm_master="ubslurm1"
namegrp="abacus"
num_nodes=4
mtype='n1-highcpu-16'
master_exists=True 
compute_nodes="bravo[1-10]"

In [34]:
def launch_cluster(namegrp, num_nodes, mtype, start_id=1, slurm_master=None):
    machine = machine_type(mtype)
    
    compute_nodes = namegrp+"[1-"+str(num_nodes*3)+"]"
    #*3 is just a margin for safety in case we fail and have to rerun the job without reconfiguring master 
    configstr = generate_slurm_config(slurm_master, machine, compute_nodes)
    cnodes_launched = []
    if slurm_master is None:
        print("Master must be running, not supported yet, exiting")
        return
    
    for i in range(start_id, num_nodes+start_id):
        #TODO: add the startid 
        name = namegrp+str(i)
        start_instance(mtype, zone, name, get_startup_script(configstr))
        cnodes_launched.append(name)
        time.sleep(5)
    print("Cluster Launched")
    return (slurm_master, cnodes_launched)

In [41]:
def run_job(slurm_master, num_nodes, machine):
    cores = machine['cores']
    runfile = "sb_confinement.sh"
    simparams = ""
    sbatcmd = "sbatch --parsable -N {num_nodes} -c {cores} -n {num_nodes} {runfile} {simparams}".format(\
        num_nodes=num_nodes, cores=cores, runfile=runfile, simparams=simparams)
    
    #ssh into the servers and then start the batch script that takes argument:
    #number of nodes, job name 
    #sbatch sb_confinement -N 
    #should also set a trigger for sure 
    #and return the job name 
    #sbatch --parsable -N 2 -c 4 -n 2 sb_confinement.sh 
    #We should then be listning for the trigger acton 
    return sbatcmd

In [43]:
run_job(None, 4, machine_type('n1-highcpu-8'))

'sbatch --parsable -N 4 -c 4 -n 4 sb_confinement.sh '

In [16]:
def reconfig_master():
    """ Copy config file, then restart slurmctld . Much easier if this is local"""
    pass 

In [35]:
range(5,9)

[5, 6, 7, 8]

In [45]:
launch_cluster('abbey', 4, 'n1-highcpu-8', 5, 'ubslurm1')

{'machineType': 'zones/us-east1-b/machineTypes/n1-highcpu-8', 'name': 'abbey5', 'disks': [{'initializeParams': {'sourceImage': 'global/images/ubs2'}, 'boot': True, 'autoDelete': True}], 'scheduling': {'preemptible': 'true'}, 'metadata': {'items': [{'value': '\n#!/bin/bash\n\nlogger "Startup Script Begins.... "\nlogger "Running as `whoami`" \n\n#systemctl stop slurmd \n\necho > /etc/slurm-llnl/slurm.conf\n\ncat <<\\EOF >> /etc/slurm-llnl/slurm.conf\n\n# slurm.conf file generated by configurator easy.html.\n# Put this file on all nodes of your cluster.\n# See the slurm.conf man page for more information.\n#\nControlMachine=ubslurm1\nAuthType=auth/none\n#CheckpointType=checkpoint/none\n#CryptoType=crypto/munge\n#ControlAddr=\n#\n#MailProg=/bin/mail\nMpiDefault=none\n#MpiParams=ports=#-#\nProctrackType=proctrack/pgid\nReturnToService=1\nSlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid\n#SlurmctldPort=6817\nSlurmdPidFile=/var/run/slurm-llnl/slurmd.pid\n#SlurmdPort=6818\nSlurmdSpoolDir=/va

Cluster Launched


('ubslurm1', ['abbey5', 'abbey6', 'abbey7', 'abbey8'])

In [33]:
def start_instance(mtype, zone, name, startupscriptstr):
    global wordlist 
    global vmnum 
    
    
    machine_type = "zones/{}/machineTypes/{}".format(zone, mtype)
    
    instance_body={
        'name':name,
        'machineType':machine_type, 
        'scheduling':
          {
        'preemptible': 'true'
          },
        'disks': [
        {
            'boot': True,
            'autoDelete': True,
            'initializeParams': {
                'sourceImage': "global/images/ubs2"
                }
        }],
        'metadata' : {
            "items" : [
                {
                    "key": "startup-script",
                    "value":startupscriptstr
                }
            ]
        },
        'networkInterfaces': [{
        'network': 'global/networks/default',
        'accessConfigs': [
            {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
        ]
    }],
    }
    print(str(instance_body))
    
    response = compute.instances().insert(project=project,zone=zone,body=instance_body).execute()
    return response 
    

In [24]:
#machine={'sockets': 1, 'cores': 1, 'threads': 2, 'memory': 1000}

In [38]:
def generate_slurm_config(slurm_master, machine, compute_nodes):
    slurmconfstr = """
# slurm.conf file generated by configurator easy.html.
# Put this file on all nodes of your cluster.
# See the slurm.conf man page for more information.
#
ControlMachine={slurm_master}
AuthType=auth/none
#CheckpointType=checkpoint/none
#CryptoType=crypto/munge
#ControlAddr=
#
#MailProg=/bin/mail
MpiDefault=none
#MpiParams=ports=#-#
ProctrackType=proctrack/pgid
ReturnToService=1
SlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid
#SlurmctldPort=6817
SlurmdPidFile=/var/run/slurm-llnl/slurmd.pid
#SlurmdPort=6818
SlurmdSpoolDir=/var/lib/slurm-llnl/slurmd
SlurmUser=slurm
#SlurmdUser=root
StateSaveLocation=/var/lib/slurm-llnl/slurmctld
SwitchType=switch/none
TaskPlugin=task/none
JobCredentialPrivateKey=/home/prateek3_14/slurmkey
#
JobCompType=jobcomp/filetxt
JobCompLoc=/var/log/slurmjobs
DebugFlags=NO_CONF_HASH
# TIMERS
#KillWait=30
#MinJobAge=300
#SlurmctldTimeout=120
#SlurmdTimeout=300
#
#
# SCHEDULING
FastSchedule=1
SchedulerType=sched/builtin
#SchedulerPort=7321
SelectType=select/linear
#
#
# LOGGING AND ACCOUNTING
AccountingStorageType=accounting_storage/none
ClusterName=ubslurm1
#JobAcctGatherFrequency=30
JobAcctGatherType=jobacct_gather/none
#SlurmctldDebug=3
SlurmctldLogFile=/var/log/slurm-llnl/slurmctld.log
#SlurmdDebug=3
SlurmdLogFile=/var/log/slurm-llnl/slurmd.log
#
#
# COMPUTE NODES \n""".format(slurm_master=slurm_master)

    slurmconfstr += ' '.join(("NodeName=DEFAULT",
                          "Sockets="        + str(machine['sockets']),
                          "CoresPerSocket=" + str(machine['cores']),
                          "ThreadsPerCore=" + str(machine['threads']),
                          "State=UNKNOWN"))

    slurmconfstr += '\n'
    #slurmconfstr += 'NodeName={slurm_master} \n'.format(slurm_master=slurm_master)
    slurmconfstr += 'NodeName={compute_nodes} \n'.format(compute_nodes=compute_nodes)


    slurmconfstr += "\n PartitionName=long Nodes={compute_nodes} Default=YES MaxTime=INFINITE State=UP \n".format(compute_nodes=compute_nodes)
    return slurmconfstr

In [26]:
#print slurmconfstr

In [31]:
def get_startup_script(slurmconfstr):
    """ Copy slurm config, and start slurmd. TODO: if master, then slurmctld"""
    
    startupscriptstr = """
#!/bin/bash

logger "Startup Script Begins.... "
logger "Running as `whoami`" 

#systemctl stop slurmd 

echo > /etc/slurm-llnl/slurm.conf

cat <<\EOF >> /etc/slurm-llnl/slurm.conf
{slurmconfstr}
EOF

systemctl start slurmd 

logger "Slurm conf applied, startup script ending" 

exit 0

""".format(slurmconfstr=slurmconfstr)
    return startupscriptstr

In [30]:
#print(startupscriptstr)

In [13]:
project='first-220321'
ze='us-east1-b'
vmnum=0

In [14]:
compute = googleapiclient.discovery.build('compute', 'v1')

In [15]:
compute.instances().list(project=project,zone=ze).execute()

{u'id': u'projects/first-220321/zones/us-east1-b/instances',
 u'items': [{u'cpuPlatform': u'Unknown CPU Platform',
   u'creationTimestamp': u'2019-02-18T11:37:09.182-08:00',
   u'deletionProtection': False,
   u'disks': [{u'autoDelete': True,
     u'boot': True,
     u'deviceName': u'persistent-disk-0',
     u'guestOsFeatures': [{u'type': u'VIRTIO_SCSI_MULTIQUEUE'}],
     u'index': 0,
     u'interface': u'SCSI',
     u'kind': u'compute#attachedDisk',
     u'licenses': [u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1804-lts'],
     u'mode': u'READ_WRITE',
     u'source': u'https://www.googleapis.com/compute/v1/projects/first-220321/zones/us-east1-b/disks/aardvark15',
     u'type': u'PERSISTENT'}],
   u'id': u'2048871841028028299',
   u'kind': u'compute#instance',
   u'labelFingerprint': u'42WmSpB8rSM=',
   u'machineType': u'https://www.googleapis.com/compute/v1/projects/first-220321/zones/us-east1-b/machineTypes/n1-standard-1',
   u'metadata': {

In [17]:
mtype='n1-standard-2'
zone=ze 
name="bravo5"

In [19]:
#d

In [20]:
response = start_instance(mtype,ze,name)

{'machineType': 'zones/us-east1-b/machineTypes/n1-standard-2', 'name': 'bravo5', 'disks': [{'initializeParams': {'sourceImage': 'global/images/ubs2'}, 'boot': True, 'autoDelete': True}], 'scheduling': {'preemptible': 'true'}, 'metadata': {'items': [{'value': '#!/bin/bash\n\nlogger "Startup Script Begins.... "\nlogger "Running as `whoami`" \n\n#systemctl stop slurmd \n\ncat <<\\EOF >> /etc/slurm-llnl/slurm.conf\n\n# slurm.conf file generated by configurator easy.html.\n# Put this file on all nodes of your cluster.\n# See the slurm.conf man page for more information.\n#\nControlMachine=ubslurm1\nAuthType=auth/none\n#CheckpointType=checkpoint/none\n#CryptoType=crypto/munge\n#ControlAddr=\n#\n#MailProg=/bin/mail\nMpiDefault=none\n#MpiParams=ports=#-#\nProctrackType=proctrack/pgid\nReturnToService=1\nSlurmctldPidFile=/var/run/slurm-llnl/slurmctld.pid\n#SlurmctldPort=6817\nSlurmdPidFile=/var/run/slurm-llnl/slurmd.pid\n#SlurmdPort=6818\nSlurmdSpoolDir=/var/lib/slurm-llnl/slurmd\nSlurmUser=slu